In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install datasets
! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install Pillow
! pip install pytesseract
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git' 
!python -m pip install torchvision tesseract
!git clone https://github.com/facebookresearch/detectron2.git
!python -m pip install -e detectron2
!pip3 install labelbox[data]
!pip install openai


In [ ]:
# model train and predict libraries
from datasets import load_dataset 
from PIL import Image, ImageDraw, ImageFont
import ast
from transformers import LayoutLMv2Processor
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D
from transformers import pipeline
from torch.utils.data import DataLoader
from transformers import LayoutLMv2ForTokenClassification, AdamW
import torch
from tqdm.notebook import tqdm # 순회 가능한 객체의 진행률을 progress bar로 표현해주고 남은 시간 정보 알려줌
import pandas as pd



In [ ]:
from datasets import load_dataset, filesystems, DatasetDict

data = load_dataset("csv", data_files='train_data.csv', split="train")  # path to your file


# Split into 70% train, 30% test + validation
train_test_validation = data.train_test_split(test_size=0.2)

# Split 30% test + validation into half test, half validation
test_validation = train_test_validation["test"].train_test_split(test_size=0.5)

# Gather the splits  to have a single DatasetDict

dataset = DatasetDict(
    {
        "train": data,
        "validation": test_validation["train"],
        "test": data[10],
    }
)
dataset['train'].save_to_disk('./train_data')

In [ ]:
# 'LayoutLMv2Processor' 을 이용하여서 이전까지 전처리 했던 데이터를 이용하여 모델 학습에 맞는 형태로 전처리를 한번 더 해준다.

from datasets import load_metric, load_from_disk,load_dataset,Features, Sequence, ClassLabel, Value, Array2D, Array3D


#labels=> 예측값의 layout 명칭
labels = ['others','key','value','total','column_name','item','count','money']
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}

#huggingface : https://huggingface.co/microsoft/layoutlmv2-base-uncased 의 pretrained 된 모델을 사용
#입력된 데이터를 모델에 학습을 시킬수 있는 데이터로 전환하는 전처리 하기 위한 모델이다.

processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")


# 전처리 데이터의 데이터 타입을 지정 
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels)),
})

def preprocess_data(example):
    processor.feature_extractor.apply_ocr=False
    images=[Image.open("/content/drive/MyDrive/클컴/train_data_image/"+i).convert('RGB') for i in example['image_path']]
    words = [ast.literal_eval(i) for i in example['words']]
    boxes = [ast.literal_eval(i) for i in example['bboxes']]
    word_labels = [ast.literal_eval(i) for i in example['ner_tags']]
    encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,
                                 padding="max_length", truncation=True)
    
    
    return encoded_inputs
dataset = load_from_disk('./train_data')
train_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset.column_names,features=features)
train_dataset.set_format(type = 'torch')
